## 演習

この演習では、Claude を使って Anthropic の研究論文を文字起こしして要約します。`images` フォルダの中に `research_paper` フォルダがあり、論文のスクリーンショットが 5 枚入っています。すでに 5 枚の画像パスをリストで用意してあります：

In [1]:
research_paper_pages = [
    "../images/research_paper/page1.png",
    "../images/research_paper/page2.png",
    "../images/research_paper/page3.png",
    "../images/research_paper/page4.png",
    "../images/research_paper/page5.png"
    ]

### 課題

Claude を使って次を行ってください：

* 研究論文画像 5 枚それぞれのテキストを文字起こしする
* 各ページのテキストを結合して、1 つの大きな全文（transcription）にする
* その全文を Claude に渡し、非技術者向けに論文全体を要約してもらう

出力例は例えば次のようになります：

> This paper explores a new type of attack on large language models (LLMs) like ChatGPT, called "Many-shot Jailbreaking" (MSJ). As LLMs have recently gained the ability to process much longer inputs, this attack takes advantage of that by showing the AI hundreds of examples of harmful or undesirable behavior. The researchers found that this method becomes increasingly effective as more examples are given, following a predictable pattern.
>
> The study tested MSJ on several popular AI models and found it could make them produce harmful content they were originally designed to avoid. This includes things like violent or sexual content, deception, and discrimination. The researchers also discovered that larger AI models tend to be more susceptible to this type of attack, which is concerning as AI technology continues to advance.
>
> The paper also looked at potential ways to defend against MSJ attacks. They found that current methods of training AI to be safe and ethical (like supervised learning and reinforcement learning) can help somewhat, but don't fully solve the problem. The researchers suggest that new approaches may be needed to make AI models truly resistant to these kinds of attacks. They emphasize the importance of continued research in this area to ensure AI systems remain safe and reliable as they become more powerful and widely used.

より良い結果を得るために、5 枚まとめて 1 回で処理するよりも、各ページを別リクエストで要約していくのがおすすめです。


In [2]:
from anthropic import Anthropic
from dotenv import load_dotenv
from IPython.display import Image

load_dotenv()

client = Anthropic()

model = "claude-sonnet-4-20250514"

In [3]:
# Image(research_paper_pages[4])

In [4]:
# 画像をbase64に変換して、image_blockにする
import base64
import mimetypes

def create_image_message(image_path):
    # Open the image file in "read binary" mode
    with open(image_path, "rb") as image_file:
        # Read the contents of the image as a bytes object
        binary_data = image_file.read()

    # Encode the binary data using Base64 encoding
    base64_encoded_data = base64.b64encode(binary_data)

    # Decode base64_encoded_data from bytes to a string
    base64_string = base64_encoded_data.decode('utf-8')

    # Get the MIME type of the image based on its file extension
    mime_type, _ = mimetypes.guess_type(image_path)

    # Create the image block
    image_block = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": mime_type,
            "data": base64_string
        }
    }

    return image_block

In [5]:
# 全ての画像をimage_blockに変換
image_blocks = [create_image_message(paper) for paper in research_paper_pages]

In [6]:
# Image_blockを渡して、テキストを文字起こしする
def image_transcription(image_block,model):
    message = [
        {
            "role": "user",
            "content": [
                image_block,
                {"type": "text", "text": "Please transcribe the text in this image. Output it accurately without omitting anything."}
            ]
        }
    ]
    response = client.messages.create(
        model=model,
        max_tokens=8192,
        messages=message
    )
    return response.content[0].text


In [7]:
# transcription = image_transcription(image_blocks[0],model)
# print(transcription)

In [8]:
# transcriptions = [image_transcription(image_block,model) for image_block in image_blocks]
transcriptions = []
for index, image_block in enumerate(image_blocks):
    print(f"Transcribing page {research_paper_pages[index]}")
    transcriptions.append(image_transcription(image_block,model))

Transcribing page ../images/research_paper/page1.png
Transcribing page ../images/research_paper/page2.png
Transcribing page ../images/research_paper/page3.png
Transcribing page ../images/research_paper/page4.png
Transcribing page ../images/research_paper/page5.png


In [9]:
# print(transcriptions[4])

In [10]:
# Image_blockを渡して、テキストを要約する
def image_summarize(image_block,model):
    message = [
        {
            "role": "user",
            "content": [
                image_block,
                {"type": "text", "text": "summarize this image."}
            ]
        }
    ]
    response = client.messages.create(
        model=model,
        max_tokens=4096,
        messages=message
    )
    return response.content[0].text


In [11]:
# summary = image_summarize(image_blocks[1],model)
# print(summary)

In [12]:
# summaries = [image_summarize(image_block,model) for image_block in image_blocks]
summaries = []
for index, image_block in enumerate(image_blocks):
    print(f"Summarizing page {research_paper_pages[index]}")
    summaries.append(image_summarize(image_block,model))

Summarizing page ../images/research_paper/page1.png
Summarizing page ../images/research_paper/page2.png
Summarizing page ../images/research_paper/page3.png
Summarizing page ../images/research_paper/page4.png
Summarizing page ../images/research_paper/page5.png


In [13]:
# print(summaries[4])

In [14]:
all_transcriptions = "\n".join(transcriptions)
all_summaries = "\n".join(summaries)

In [15]:
# print(all_transcriptions)
# print(all_summaries)

In [16]:
def final_summary(summary:str,transcription:str, model:str):
    prompt = f"""
    # 指示
        あなたは研究論文を要約する、頼りになるアシスタントです。
        研究論文の要約と書き起こしが提供されます。
        技術に詳しくない読者にも分かりやすいように、研究論文を要約してください。
    # 要約
        {summary}
    # 書き起こし
        {transcription}
    # 出力
        出力は、日本語で2000文字以内にしてください。
        １行は100文字以内にしてください。
    """
    message = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    response = client.messages.create(
        model=model,
        max_tokens=4096,
        messages=message
    )
    return response.content[0].text


In [17]:
print("Final Summarizing...")
final_summary = final_summary(all_summaries,all_transcriptions,model)

Final Summarizing...


In [18]:
print(final_summary)

# 「多ショット・ジェイルブレイキング」研究の分かりやすい解説

## 研究の概要

この研究は、最新のAI（大規模言語モデル）に対する新しいタイプの攻撃手法について調べたものです。
「多ショット・ジェイルブレイキング（MSJ）」と呼ばれるこの手法は、AIの安全対策を回避して、
本来なら拒否すべき有害な質問に答えさせる攻撃です。

## 攻撃の仕組み

従来の攻撃では少数の例しか使えませんでしたが、最近のAIは長い文章を理解できるように
なったため、数百もの有害な質問と回答の例を一度に見せることが可能になりました。
例えば、「爆弾の作り方を教えて」のような危険な質問に対して、AIが協力的に答えている
会話例を大量に提示することで、AIをだまして同様の有害な回答をさせる手法です。

## 主な発見

**効果の高さ**: Claude 2.0、GPT-3.5、GPT-4、Llama 2など、主要なAIモデル全てが
この攻撃に弱いことが分かりました。

**規則的なパターン**: 攻撃の成功率は「べき法則」と呼ばれる数学的なパターンに従って
増加することが判明しました。これにより、どれくらいの例を示せば攻撃が成功するかを
予測できるようになりました。

**大きなモデルほど危険**: より高性能なAIほど、この攻撃に対して脆弱であることが
明らかになりました。

## 防御対策の限界

研究チームは様々な防御手法を試しましたが、根本的な解決策は見つかりませんでした。

**従来の安全訓練の限界**: 現在広く使われている「人間のフィードバックによる強化学習」
などの手法では、攻撃に必要な例の数を増やすことはできても、完全に防ぐことはできません。

**対象特化型の訓練も不十分**: この攻撃に特化した防御訓練を行っても、
十分に長い文脈での攻撃は防げませんでした。

## 社会への影響

この研究は、AIの安全性に関する重要な警告を発しています。AIがより高性能になり、
より長い文章を処理できるようになることで、新たな脆弱性が生まれる可能性があります。

**開発者への警鐘**: AI開発者は、新機能を追加する際に意図しない攻撃の機会を
作り出していないか、慎重に検討する必要があります。

**今後の研究の必要性**: この攻撃を根本的に防ぐためには、現在の安全対策を


個別確認用スニペット

In [19]:
# message = [
#     {
#         "role": "user",
#         "content": [
#             image_blocks[0],
#             {"type": "text", "text": "このページのテキストを日本語で文字起こししてください"}
#         ]
#     }
# ]
# response = client.messages.create(
#     model="claude-sonnet-4-20250514",
#     max_tokens=4096,
#     messages=message
# )
# print(response.content[0].text)
